In [11]:
import pennylane as qml
from pennylane import numpy as np
import math
from pennylane.optimize import AdamOptimizer

In [12]:
num_qubit_factoring = 10+1
dev1 = qml.device('default.qubit', wires=num_qubit_factoring)
@qml.qnode(dev1)
def oracle_circuit():
    for i in range(1,5):
      qml.Hadamard(wires=i)  

     
    #C1
    qml.CNOT(wires=[1,3])
    qml.CNOT(wires=[3,5])
    qml.CNOT(wires=[1,3])
    
    #C2
    qml.CNOT(wires=[2,4])
    qml.CNOT(wires=[4,6])
    qml.CNOT(wires=[2,4])
    
    #AND1
    qml.Toffoli(wires=[5,6,7])
    
    #C3
    qml.Toffoli(wires=[1,4,8])
    qml.Toffoli(wires=[2,3,9])
    qml.CNOT(wires=[8,9])
    
    #AND2
    qml.Toffoli(wires=[7,9,0])
    
    return qml.probs([i for i in range(0,5)])
    

oracle_result = oracle_circuit()
norm = np.linalg.norm(oracle_result)
oracle_result /= norm
oracle_result = oracle_result.tolist()
print(oracle_result)

print('Oracle:')
print(qml.draw(oracle_circuit)())

[0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.0, 0.25, 0.25, 0.0, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Oracle:
0: ───────────────────────────────╭X─┤ ╭Probs
1: ──H─╭●────╭●──────────╭●───────│──┤ ├Probs
2: ──H─│─────│──╭●────╭●─│──╭●────│──┤ ├Probs
3: ──H─╰X─╭●─╰X─│─────│──│──├●────│──┤ ├Probs
4: ──H────│─────╰X─╭●─╰X─├●─│─────│──┤ ╰Probs
5: ───────╰X───────│──╭●─│──│─────│──┤       
6: ────────────────╰X─├●─│──│─────│──┤       
7: ───────────────────╰X─│──│─────├●─┤       
8: ──────────────────────╰X─│──╭●─│──┤       
9: ─────────────────────────╰X─╰X─╰●─┤       


In [13]:
def vqs_factoring(num_qubits_vqs,n_layers,oracle,n_shots=None,device_name1='default.qubit' ,device_name2='default.qubit' ):
    print('run VQS n_layers=',n_layers,'device1 = ',device_name1,'device2 = ',device_name2)
    val_global = []
    # vqs
    num_qubit_vqs = 1+num_qubits_vqs
    eps_val_q = 1/math.sqrt(2**num_qubit_vqs)/100
    eps_val = min(1e-10, eps_val_q)
    tiny_change_threshold = 1e-4
    cnt_threshold_no_change = 5
    

    N = 2**(num_qubit_vqs-2)
    normal_val = math.sqrt(1/N)


    def oracle_builder_for_HT_HTZ():
        initial_state_0_phi1  = oracle_result + [0]*len(oracle_result) 
        qml.QubitStateVector(np.array(initial_state_0_phi1), wires=range(num_qubits_vqs))

    def oracle_builder_for_no_HT_HTZ():
        qml.QubitStateVector(np.array(oracle), wires=range(num_qubits_vqs-1))


    def layer_t3_no_HT(theta, qubit_posi):
        # type-2 layer
        # length of theta: (num_qubit_vqs-1)*2
        # length of qubit_posi: num_qubit_vqs-1
        # number of wires: num_qubit_vqs
        for i in range(num_qubit_vqs-1):
            qml.RY(theta[i], wires=(qubit_posi[i]))
        for i in np.arange(0, num_qubit_vqs-2, 2):
            qml.CNOT(wires=(qubit_posi[i], qubit_posi[i+1]))  # CNOT struct3
        for i in range(num_qubit_vqs-1):
            qml.RY(theta[i+num_qubit_vqs-1], wires=(qubit_posi[i]))
        for i in np.arange(1, num_qubit_vqs-2, 2):
            qml.CNOT(wires=(qubit_posi[i], qubit_posi[i+1]))  # CNOT struct3
        qml.CNOT(wires=(qubit_posi[-1], qubit_posi[0]))  # CNOT struct3


    def layer_t3_with_HT(theta, num_qubit_vqs):
        # type-2 layer
        # length of theta: (num_qubit_vqs-1)*2
        # number of wires: num_qubit_vqs
        for i in range(num_qubit_vqs-1):
            qml.CRY(theta[i], wires=(0, i+1))
        for i in np.arange(0, num_qubit_vqs-2, 2):
            qml.Toffoli(wires=(0, i+1, i+2))  # CCNOT struct3

        for i in range(num_qubit_vqs-1):
            qml.CRY(theta[i+num_qubit_vqs-1], wires=(0, i+1))
        for i in np.arange(1, num_qubit_vqs-2, 2):
            qml.Toffoli(wires=(0, i+1, i+2))  # CCNOT struct3
        qml.Toffoli(wires=(0, num_qubit_vqs-1, 1))  # CCNOT struct3


    dev_with_HT = qml.device(device_name1, wires=num_qubits_vqs+1,shots=n_shots)
    @qml.qnode(dev_with_HT)
    def quantum_circuit_with_HT(theta):
        oracle_builder_for_HT_HTZ()
        qml.Hadamard(0)
        for theta_i in theta:
            layer_t3_with_HT(theta_i, num_qubit_vqs)
        qml.Hadamard(0)
        
        return qml.expval(qml.PauliZ(0))

    dev_with_HTZ = qml.device(device_name1, wires=num_qubits_vqs+1,shots=n_shots)
    @qml.qnode(dev_with_HTZ)
    def quantum_circuit_with_HTZ(theta):
        oracle_builder_for_HT_HTZ()
        qml.Hadamard(0)
        for theta_i in theta:
            layer_t3_with_HT(theta_i, num_qubit_vqs)
        qml.CZ([0, 1])
        qml.Hadamard(0)
        return qml.expval(qml.PauliZ(0))


    dev_no_HT_S = qml.device(device_name2, wires=num_qubits_vqs,shots=n_shots)
    @qml.qnode(dev_no_HT_S)
    def quantum_circuit_no_HT_return_state(theta):
        oracle_builder_for_no_HT_HTZ()
        for theta_i in theta:
            layer_t3_no_HT(theta_i, list(range(num_qubit_vqs-1)))
        return qml.state()


    def objective_fn(theta):
        val1_1 = quantum_circuit_with_HT(theta)
        val1_2 = quantum_circuit_with_HTZ(theta)
        val1_1 = val1_1/normal_val
        val1_2 = val1_2/normal_val
        obj = -0.5*(val1_1 - val1_2)
        val_global.append(
            [val1_1._value.tolist(), val1_2._value.tolist(), obj._value.tolist()])
        
        return obj

    iter_max = 300  # 300
    num_of_layers = n_layers
    obj_list_rep = []
    theta_list = []
    iter_terminate_list = []
    print_flag = True
    optimizer = AdamOptimizer(0.05, beta1=0.9, beta2=0.999)
    theta = np.random.uniform(
        0, 2*math.pi, size=(num_of_layers, 2*(num_qubit_vqs-1)), requires_grad=True)

    obj_list = []
    tiny_change_cnt = 0
    break_flag = False
    iter_terminate = iter_max
    for iter in range(1, iter_max+1):
        theta, obj = optimizer.step_and_cost(objective_fn, theta)
        val1_1 = val_global[-1][0]
        val1_2 = val_global[-1][1]
        if iter >= 2:
            val1_1_old = val_global[-2][0]
            val1_2_old = val_global[-2][1]
        else:
            val1_1_old = 999
            val1_2_old = 999
        val1 = val1_1 - val1_2
        val1_old = val1_1_old - val1_2_old
        if abs(val1) > eps_val:  # eps_val=1e-10
            if abs((val1-val1_old)/val1) < tiny_change_threshold:  # 1e-3
                tiny_change_cnt += 1
            else:
                tiny_change_cnt = 0
        if tiny_change_cnt >= cnt_threshold_no_change:  # no change for a consequtive of 5 iterations, then break
            break_flag = True
        if (iter == 1 or iter % 10 == 0 or iter == iter_max) and print_flag:
            #TODO ? 
            if (np.isclose(val1_1-val1_2,0)):
                return False
            print(f'iter={iter:3d} :: obj={obj:12.8f} :: val1_1={val1_1:12.8f} :: val1_2={val1_2:12.8f} :: -0.5*(val1_1 - val1_2)={-0.5*(val1_1 - val1_2):12.8f}')

        obj_list.append(obj)
        if break_flag:
            iter_terminate = iter
            break
    theta_list.append(theta)
    obj_list_rep.append(obj_list)

    val_global = []  # reset to empty

    # display the amplified state
    state = quantum_circuit_no_HT_return_state(theta)
    iter_terminate_list.append(iter_terminate)
    return state

In [14]:
num_qubits_vqs = 5+1
n_layers = 3
results = vqs_factoring(num_qubits_vqs=num_qubits_vqs,n_layers=n_layers,oracle=oracle_result)

run VQS n_layers= 3 device1 =  default.qubit device2 =  default.qubit
iter=  1 :: obj=  0.24554700 :: val1_1= -0.37724128 :: val1_2=  0.11385272 :: -0.5*(val1_1 - val1_2)=  0.24554700
iter= 10 :: obj= -1.29774341 :: val1_1=  1.33500729 :: val1_2= -1.26047953 :: -0.5*(val1_1 - val1_2)= -1.29774341
iter= 20 :: obj= -1.71068027 :: val1_1=  1.59779926 :: val1_2= -1.82356127 :: -0.5*(val1_1 - val1_2)= -1.71068027
iter= 30 :: obj= -1.85472101 :: val1_1=  1.78212261 :: val1_2= -1.92731941 :: -0.5*(val1_1 - val1_2)= -1.85472101
iter= 40 :: obj= -1.88555546 :: val1_1=  1.90837934 :: val1_2= -1.86273158 :: -0.5*(val1_1 - val1_2)= -1.88555546
iter= 50 :: obj= -1.90778663 :: val1_1=  1.87497851 :: val1_2= -1.94059475 :: -0.5*(val1_1 - val1_2)= -1.90778663
iter= 60 :: obj= -1.91716076 :: val1_1=  1.87367888 :: val1_2= -1.96064264 :: -0.5*(val1_1 - val1_2)= -1.91716076
iter= 70 :: obj= -1.92342547 :: val1_1=  1.83058439 :: val1_2= -2.01626655 :: -0.5*(val1_1 - val1_2)= -1.92342547
iter= 80 :: obj= -

In [15]:
results_probs = [(np.linalg.norm(res))**2 for res in results]
finded_solutions={}
for i in range(len(results_probs)):
    if(results_probs[i]>0.1):
        binary_i = '{0:06b}'.format(i)
        finded_solutions[binary_i[1:5]]=results_probs[i]
print(finded_solutions)
counter = 0
for key in finded_solutions.keys():
    if counter!=0:
        print('OR')
    print(fr'p1={key[0]} & p2={key[1]} & q1={key[2]} & q2={key[3]}')
    counter+=1

{'0110': 0.47961978783684156, '1001': 0.4796978075594804}
p1=0 & p2=1 & q1=1 & q2=0
OR
p1=1 & p2=0 & q1=0 & q2=1
